# Data Validation with Great Expectations

# Create Data Context
Pertama definisikan terlebih dahulu dimana data context atau tempat expectations validasi data disimpan. Pada program ini, data context akan disimpan pada folder `gx_data_context`, dan karena menggunakan dua titik `".."` maka folder `gx_data_context` akan dibuat pada parent directory, bukan directory yang selevel dengan program ini.

In [ ]:
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='../gx_data_context')

# Connect to data source
Lalu datasource akan dikoneksikan dan disiapkan.

In [ ]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-data-shipping'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'shipping-january'
path_to_data = 'data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Create Expectation Suite
Lalu expectation suite akan di buat, di mana berbagai expectation akan disimpan pada expectation suite ini.

In [3]:
# Creat an expectation suite
expectation_suite_name = 'expectation-shipping-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,id,warehouse_block,mode_of_shipment,customer_care_calls,customer_rating,cost_of_the_product,prior_purchases,product_importance,gender,discount_offered,weight_in_gms,reached_on_time_y_n
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


## Expectations

### Expectation 1
Kolom **'id'** merupakan **'id'** dari shipping pemesanan tersebut. Maka dari itu shipping pemesanan tersebut harus memiliki **'id'** yang berbeda agar dapat diketahui spesifik pemesanan mana yang sedang direpresentasikan. Maka dari itu `kolom 'id' harus bernilai unique`

In [5]:
# Expectation 1 : Kolom `id` harus unique

validator.expect_column_values_to_be_unique('id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 2
Expectation 2 yang akan digunakan adalah to be between min value dan max value, yang berarti nilai dari sebuah data pada kolom tertentu tersebut harus dalam jangkauan tertentu. Terdapat 2 kolom yang akan diterapkan expectation ini yaitu:
* Kolom **'customer_rating'** karena rating dari pelanggan hanya dapat memiliki jangkauan dari 1 hingga 5 saja tidak lebih dan tidak kurang
* Kolom **'discount_offered'** karena diskon yang diberikan oleh tidak mungkin kurang dari 0 dan tidak mungkin lebih dari 99.

In [6]:
# Expectation 2, column `customer_rating` : Kolom `customer_rating` harus dalam jangkauan 1 hingga 5

validator.expect_column_values_to_be_between(
    column='customer_rating', min_value=1, max_value=5
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [7]:
# Expectation 2, column `discount_offered` : Kolom `discount_offered` harus dalam jangkauan 0 hingga 99

validator.expect_column_values_to_be_between(
    column='discount_offered', min_value=0, max_value=99
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 3
Expectation 3 yang akan diterapkan adalah to be in set, yang berarti value dari sebuah kolom hanya dapat berada dalam suatu set tertentu yang sudah ditetapkan. Terdapat 4 kolom yang akan diterapkan expectation ini yaitu:
* Kolom **'mode_of_shipment'** karena saat ini SwiftCart hanya menyediakan pengiriman 'Flight','Ship', dan 'Road' saja.
* Kolom **'product_importance'** karena saat ini SwiftCart hanya mengkategorikan suatu pengiriman dalam 3 kategori saja yaitu 'low', 'medium', dan 'high'
* Kolom **'gender'** karena hanya ada 2 gender yaitu laki-laki dan perempuan. Dalam dataset ini direpresentasikan sebagai 'M' dan 'F'
* Kolom **'reached_on_time_y_n'** karena hanya ada dua kemungkinan dari pengiriman yang telah dilakukan yaitu terlambat atau tidak terlambat. Ini direpresentasikan dalam angka 0 dan 1, dimana 1 berarti terlambat dan 0 berarti tepat waktu.

In [8]:
# Expectation 3, column `mode_of_shipment` : Kolom `mode_of_shipment` hanya dapat berisi ['Flight', 'Ship', 'Road']

validator.expect_column_values_to_be_in_set('mode_of_shipment', ['Flight', 'Ship', 'Road'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [9]:
# Expectation 3, column `product_importance` : Kolom `product_importance` hanya dapat berisi ['low', 'medium', 'high']

validator.expect_column_values_to_be_in_set('product_importance', ['low', 'medium', 'high'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
# Expectation 3, column `gender` : Kolom `gender` hanya dapat berisi ['M', 'F']

validator.expect_column_values_to_be_in_set('gender', ['M', 'F'])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [11]:
# Expectation 3, column `reached_on_time_y_n` : Kolom `reached_on_time_y_n` hanya dapat berisi [0, 1]

validator.expect_column_values_to_be_in_set('reached_on_time_y_n', [0, 1])

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 4
Expectation 4 yang akan digunakan adalah to be of type yang berarti kolom tersebut hanya boleh memiliki tipe data yang telah ditentukan. Terdapat 3 kolom yang akan diterapkan expectation ini yaitu:

* Kolom **'customer_care_calls'** karena kolom ini bersifat numerikal dan diskrit, maka hanya diperbolehkan bernilai integer
* Kolom **'customer_rating'** karena kolom ini bersifat numerikal dan diskrit, maka hanya diperbolehkan bernilai integer
* Kolom **'prior_purchases'** karena kolom ini bersifat numerikal dan diskrit, maka hanya diperbolehkan bernilai integer

In [12]:
# Expectation 4, column `customer_care_calls` : Kolom `customer_care_calls` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_of_type('customer_care_calls', 'int64')

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
# Expectation 4, column `customer_rating` : Kolom `customer_rating` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_of_type('customer_rating', 'int64')

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
# Expectation 4, column `prior_purchases` : Kolom `prior_purchases` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_of_type('prior_purchases', 'int64')

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 5
Expectation 5 yang akan digunakan adalah to be in type list yang berarti kolom tersebut hanya boleh memiliki tipe data yang telah ditentukan dalam list. Terdapat 3 kolom yang akan diterapkan expectation ini yaitu:

* Kolom **'cost_of_the_product'** karena kolom ini bersifat numerikal dan dapat bernilai diskrit atau kontinu, maka hanya diperbolehkan bernilai integer atau float
* Kolom **'discount_offered'** karena kolom ini bersifat numerikal dan dapat bernilai diskrit atau kontinu, maka hanya diperbolehkan bernilai integer atau float
* Kolom **'weight_in_gms'** karena kolom ini bersifat numerikal dan dapat bernilai diskrit atau kontinu, maka hanya diperbolehkan bernilai integer atau float

In [15]:
# Expectation 5, column `cost_of_the_product` : Kolom `cost_of_the_product` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_in_type_list('cost_of_the_product', ['int64', 'float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
# Expectation 5, column `discount_offered` : Kolom `discount_offered` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_in_type_list('discount_offered', ['int64', 'float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [17]:
# Expectation 5, column `weight_in_gms` : Kolom `weight_in_gms` hanya dapat memiliki tipe data integer

validator.expect_column_values_to_be_in_type_list('weight_in_gms', ['int64', 'float64'])

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "int64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 6
Expectation ke 6 yang akan diterapkan adalah min to be in between yang berarti nilai minimal dari suatu kolom hanya boleh dalam jangkauan tertentu. Terdapat 4 kolom yang akan diterapkan expectation ini yaitu:

* Kolom **'customer_care_calls'** karena kolom ini tidak bisa bernilai negatif, hanya dapat bernilai 0 atau lebih. Karena tidak bisa diketahui pasti nilai maksimalnya maka nilai maksimal tidak akan didefinisikan yang berarti bisa bernilai berapapun selama tidak negatif.
* Kolom **'cost_of_the_product'** karena kolom ini tidak bisa bernilai negatif, hanya dapat bernilai lebih dari 0. Karena tidak bisa diketahui pasti nilai maksimalnya maka nilai maksimal tidak akan didefinisikan yang berarti bisa bernilai berapapun selama tidak negatif.
* Kolom **'prior_purchases'** karena kolom ini tidak bisa bernilai negatif, hanya dapat bernilai 0 atau lebih. Karena tidak bisa diketahui pasti nilai maksimalnya maka nilai maksimal tidak akan didefinisikan yang berarti bisa bernilai berapapun selama tidak negatif.
* Kolom **'weight_in_gms'** karena kolom ini tidak bisa bernilai negatif, hanya dapat bernilai lebih dari 0. Karena tidak bisa diketahui pasti nilai maksimalnya maka nilai maksimal tidak akan didefinisikan yang berarti bisa bernilai berapapun selama tidak negatif.

In [18]:
# Expectation 6 column `customer_care_calls` : Kolom `customer_care_calls` hanya dapat memiliki nilai lebih dari sama dengan 0

validator.expect_column_min_to_be_between(column="customer_care_calls", min_value=0)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
# Expectation 6 column `cost_of_the_product` : Kolom `cost_of_the_product` hanya dapat memiliki nilai lebih dari 0

validator.expect_column_min_to_be_between(column="cost_of_the_product", min_value=0, strict_min=True)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 96
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [20]:
# Expectation 6 column `prior_purchases` : Kolom `prior_purchases` hanya dapat memiliki nilai lebih dari sama dengan 0

validator.expect_column_min_to_be_between(column="prior_purchases", min_value=0)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [21]:
# Expectation 6 column `weight_in_gms` : Kolom `weight_in_gms` hanya dapat memiliki nilai lebih dari 0

validator.expect_column_min_to_be_between(column="weight_in_gms", min_value=0, strict_min=True)

Calculating Metrics:   0%|          | 0/4 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 1001
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 7
Expectation ke 7 yang akan digunakan adalah value lengths to be between yang berarti panjang string kolom hanya boleh dalam jangkauan tertentu. Kolom yang akan diterapkan expectation ini adalah kolom **'warehouse_block'**. Karena saat ini SwiftCart hanya memiliki 5 warehouse (yang dikodekan menggunakan huruf A hingga E), dan jika terdapat kemungkinan penambahan jumlah warehouse, maka jumlah minimal string adalah 1 dan jumlah maksimal string adalah 5. Ini berarti jumlah maksimal warehouse yang diekspektasikan saat ini adalah 26x5 (26 huruf alfabet dan panjang string maksimal 5, dari A - ZZZZZ) yang berarti terdapat 130 warehouse yang pada saat ini sudah lebih dari cukup.
Maka dari itu `kolom 'warehouse_block' harus memiliki panjang string dari 1 hingga 5`

In [22]:
# Expectation 8 column `warehouse_block` : Kolom `warehouse_block` hanya dapat memiliki panjang string di antara 1 hingga 5

validator.expect_column_value_lengths_to_be_between(column="warehouse_block", min_value=1, max_value=5)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 8
Expectation ke 8 yang akan diterapkan adalah value to match regex yang berarti string kolom hanya boleh sesuai regex yang ditentukan. Kolom yang akan diterapkan expectation ini adalah kolom **'warehouse_block'**. Saat ini SwiftCart hanya merepresentasikan warehouse hanya dalam huruf kapital dari A hingga Z dan tidak menggunakan tanda baca, simbol, angka, ataupun huruf kecil.
Maka dari itu `kolom 'warehouse_block' harus memiliki huruf A hingga Z saja`

In [23]:
# Expectation 8 column `warehouse_block` : Kolom `warehouse_block` hanya dapat memiliki huruf A hingga Z

validator.expect_column_values_to_match_regex(column = "warehouse_block", regex = r"^[A-Z]+$")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 10999,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 9
Expectation ke 9 yang akan diterapkan adalah column to exist yang berarti kolom tertentu diharuskan terdapat pada dataset. Kolom yang akan diterapkan expectation ini adalah kolom **'reached_on_time_y_n'** karena kolom inilah yang penting untuk diketahui apakah shipping sampai tepat waktu atau tidak dan bagian dari objektif saat ini.
Maka dari itu `kolom 'reached_on_time_y_n' harus ada pada dataset`

In [24]:
# Expectation 9 : Column `reached_on_time_y_n` harus ada pada dataset

validator.expect_column_to_exist(column='reached_on_time_y_n')

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Save expectations suite
Setelah seluruh expectations telah dilakukan di atas, expectations tersebut akan disimpan ke dalam expectations suite

In [25]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

## Checkpoint
Lalu checkpoint akan dibuat di mana checkpoint adalah kode yang menjalankan validasi data secara otomatis berdasarkan expectations suite yang telah dibuat sebelumnya.

In [26]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [29]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()
checkpoint_result

Calculating Metrics:   0%|          | 0/69 [00:00<?, ?it/s]

{
  "run_id": {
    "run_name": null,
    "run_time": "2025-10-26T00:17:22.763354+07:00"
  },
  "run_results": {
    "ValidationResultIdentifier::expectation-shipping-dataset/__none__/20251025T171722.763354Z/csv-data-shipping-shipping-january": {
      "validation_result": {
        "success": true,
        "results": [
          {
            "success": true,
            "expectation_config": {
              "expectation_type": "expect_column_values_to_be_unique",
              "kwargs": {
                "column": "id",
                "batch_id": "csv-data-shipping-shipping-january"
              },
              "meta": {}
            },
            "result": {
              "element_count": 10999,
              "unexpected_count": 0,
              "unexpected_percent": 0.0,
              "partial_unexpected_list": [],
              "missing_count": 0,
              "missing_percent": 0.0,
              "unexpected_percent_total": 0.0,
              "unexpected_percent_nonmissing":

## Data Docs
Kemudian data docs akan di buat sehingga dapat ditampilkan pada dashboard yang dapat diakses pada folder `gx_data_context` yang telah di buat pada create data context sebelumnya, tepatnya di direktori `gx_data_context/uncommitted/data_docs/index.html`

In [28]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://d:\\Bootcamp_Hacktiv8\\phase_2\\milestone_3\\p2-ftds047-rmt-m3-wesleyhakim\\..\\gx_data_context\\gx\\uncommitted/data_docs/local_site/index.html'}